In [ ]:
#Importig all the libraries

import os
import gdown
import tensorflow as tf
from deepface.commons import functions
from deepface.commons.logger import Logger
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
        Lambda,
)

In [ ]:
#VGG-Face Model
def loadModel():
    model = Sequential()
    model.add(data_augmentation)

    model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(4096, (7, 7), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation("softmax"))
    model.build((None, 224, 224, 3)) 

    model.load_weights(os.path.join("/kaggle/input","aaazzzz", "vgg_face_weights.h5"))

    model_output = Flatten()(model.layers[-5].output)
    model_output = Lambda(lambda x: K.l2_normalize(x, axis=1), name="norm_layer")(
        model_output
    )

    return Model(inputs=model.input, outputs=model_output)


data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(0.1, 0.1, fill_mode="nearest", interpolation="bilinear")
    ]
)
#Pulling the celebrity faces dataset

base_path = '/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset'
folders = os.listdir(base_path)[:10]
df = pa.DataFrame(
    [
        (os.path.join(base_path, folder, picture), folder[5:])
        for folder in folders
        for picture in os.listdir(os.path.join(base_path, folder))
    ],
    columns=['path', 'celebrity']
).assign(celebrity_encoded=lambda x: LabelEncoder().fit_transform(x['celebrity']))
folds = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X=df['path'], y=df['celebrity_encoded']))

import cv2
from matplotlib import pyplot as plt
from xgboost import XGBClassifier

def classification1 (train_embedding, test_embedding, train_labels, test_labels):
    scaler = StandardScaler()
    train_embedding = scaler.fit_transform(train_embedding)
    test_embedding = scaler.transform(test_embedding)
    pca = PCA(n_components=128)
    train_embedding = pca.fit_transform(train_embedding)
    test_embedding = pca.transform(test_embedding)
    re = RandomForestClassifier(n_jobs = -1)
    re.fit(train_embedding, train_labels)
    predict  = re.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification2 (train_embedding, test_embedding, train_labels, test_labels):
    scaler = StandardScaler()
    train_embedding = scaler.fit_transform(train_embedding)
    test_embedding = scaler.transform(test_embedding)
    pca = PCA(n_components=128)
    train_embedding = pca.fit_transform(train_embedding)
    test_embedding = pca.transform(test_embedding)
    clf = SVC(C=5., gamma=0.001)
    clf.fit(train_embedding, train_labels)
    predict = clf.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification3 (train_embedding, test_embedding, train_labels, test_labels):
    re = RandomForestClassifier(n_jobs = -1)
    re.fit(train_embedding, train_labels)
    predict  = re.predict(test_embedding)
    return accuracy_score(test_labels, predict)

def classification4 (train_embedding, test_embedding, train_labels, test_labels):
    xgb = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    xgb.fit(train_embedding, train_labels)
    predict = xgb.predict(test_embedding)
    return accuracy_score(test_labels, predict)
    
import cv2
import matplotlib.pyplot as plt

def img_to_face(img):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]
    face_region = img[y:y+h, x:x+w]
    face_region_rgb = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)
    return face_region_rgb

def test_model(model_id):
    print("Model - > VG" )
    answer = []
    fold_number = 0
    for fold in folds:
        print("Fold -> ", fold_number)
        train = df.iloc[fold[0]]
        test = df.iloc[fold[1]]
        train_labels = train["celebrity_encoded"]
        test_labels = test["celebrity_encoded"]
        
        train_embedding=[]
        test_embedding = []
        
        model = loadModel()

        for counter, (index, img) in enumerate(train.iterrows()):
            print(counter)
            
            imgg = img_to_face(cv2.imread(img['path']))
            if imgg is None:
                imgg = cv2.imread(img['path'])

            imgg = cv2.resize(imgg, (224, 224))
            imgg = np.expand_dims(imgg, axis=0)
            imgg = imgg.astype('float64') / 255.0 
            train_embedding.append( model.predict(imgg)[0, :])

        print(len(test))
        for counter, (index, img) in enumerate(test.iterrows()):
            print(counter)
            
            imgg = img_to_face(cv2.imread(img['path']))
            if imgg is None:
                imgg = cv2.imread(img['path'])
            imgg = cv2.resize(imgg, (224, 224))
            imgg = imgg.astype('float64') / 255.0 
            imgg = np.expand_dims(imgg, axis=0) 
            
            test_embedding.append( model.predict(imgg)[0, :])
        
        answer.append([classification1(train_embedding, test_embedding, train_labels, test_labels), classification2(train_embedding, test_embedding, train_labels, test_labels),classification3(train_embedding, test_embedding, train_labels, test_labels),classification4(train_embedding, test_embedding, train_labels, test_labels)])
        print("For this fold -> ", answer[fold_number])
        fold_number = fold_number + 1
    res1 = 0
    res2 = 0
    res3=0
    res4=0
    
    
    for i in range(fold_number):
        res1 += answer[i][0]
        res2 += answer[i][1]
        res3 += answer[i][2]
        res4 += answer[i][3]
    return (res1 / fold_number, res2 / fold_number, res3/ fold_number, res4/foldernumber)

def models_test():
    for i in range(0,8):
        ress = test_model(i)
        print("For model -> ", "vg", " The classification accuracy with rf + pca +ss -> " , ress[0], " accuracy with svc + pca + ss -> ", ress[1], " accuracy with normal rf -> ", ress[2] , ' XGBOST=> ' , ress[3])
    
models_test()

!PIP INSTALL ULTRALYTICS